In [1]:
import common_vars as c_vars
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import sparse
import pickle
import bisect

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv(c_vars.train_split_train_sample, nrows = 1000)

In [8]:
df.fillna(c_vars.fillna_dict, inplace = True)

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click
0,IDsrk7SoW,2017-01-14 09:42:09,4709696.0,887235,17714,20301556,e,Firefox,-999,0
1,IDVLNN0Ut,2017-01-11 12:46:49,98480.0,518539,25085,2050923,a,Edge,-999,0
2,ID32T6wwQ,2017-01-17 10:18:43,8896401.0,390352,40339,72089744,c,Firefox,Mobile,0
3,IDqUShzMg,2017-01-14 16:02:33,5635120.0,472937,12052,39507200,d,Mozilla Firefox,Desktop,0
4,IDjO9XQ1Z,2017-01-14 12:08:49,2729292.0,961176,33638,47079934,e,Google Chrome,Mobile,0
5,IDUuVWVrO,2017-01-12 13:07:08,7007059.0,664666,68847,58604466,b,Edge,-999,0
6,IDFnmhUgG,2017-01-13 05:58:13,7295565.0,144797,33638,23981625,b,Firefox,Mobile,0
7,IDmyijsRf,2017-01-14 12:38:58,5329483.0,952097,89680,74363610,a,Edge,Tablet,0
8,IDGQGYgdk,2017-01-14 11:06:26,6436389.0,949077,76239,65017662,a,Edge,Tablet,0
9,IDeS7iMpY,2017-01-12 03:36:53,6401189.0,68770,43612,74868704,b,Google Chrome,Mobile,0


In [9]:
print (sum(df['siteid'].isnull()))
print (sum(df['offerid'].isnull()))

0
0


In [12]:
df['siteid'].astype(np.int64)

0      4709696
1        98480
2      8896401
3      5635120
4      2729292
5      7007059
6      7295565
7      5329483
8      6436389
9      6401189
10     6718234
11     6313991
12      469603
13     4260541
14       48650
15     7147868
16     6510239
17     3723198
18     5914861
19     8672716
20     4989750
21     8022638
22     7721600
23     9425019
24     2368040
25     5369414
26     8672716
27     5042966
28        -999
29     1049799
        ...   
969       -999
970    1266440
971    8672432
972    5249729
973     325124
974    6026766
975    5449448
976    8967428
977    8019116
978    6252919
979    2605848
980       -999
981    3505290
982    6012391
983     470218
984    7044111
985     906159
986    3032554
987    8105045
988    5872883
989       -999
990    8174394
991    1517929
992    3796291
993    8293717
994    2673391
995    1369225
996    5632230
997    1154405
998    8703969
Name: siteid, dtype: int64